# task7_5_starter (Starter file)
## Working with a Log File

In [17]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from folium import Map, plugins
from pygeoip import GeoIP, GeoIPError

In [2]:
filepath = '../resources/new_access.log'

<br><br>
### Read the data from our the log file.  Display its shape, first few records, and column structure. 

In [2]:
# Step 1. Create a new DataFrame, call it log.
#         Use read_csv() to acquire the log file data.  To read from this non-CSV file, use sep='\s+'.
#         Also, use the following column names:  names=['addr', 'req_date', 'path', 'status', 'size', 'browser']
#         Finally, don't read all the resulting columns, lets only use these columns (indexes): 0, 3, 5, 6, 7, 9.

# Display the shape of the resulting DataFrame.


In [1]:
# Display the first few records (hint: use head() )


In [3]:
# Display the meta-data of the DataFrame (e.g., use info() )


<br><br>
### View the unique values in the *status* column.

In [4]:
# Step 2. Display the unique values found in the status column.  Try it by selecting the column and using unique()


<br><br>
### Use *value_counts()* instead to get a better picture for each status value.

In [5]:
# value_counts() gives the count values for each status.  It works the same way, try invoking value_counts() instead.


<br><br>
### Convert the date column, which is currently a string, into an actual datetime object.  View the results again.

In [6]:
# Step 3. The date shows up as a string.  This is due to the leading '[' in front of each date value.  Convert the string
#         into a datetime object using pd.to_datetime().  Pass the req_date column in and the format.  Use format='[%d/%b/%Y:%H:%M:%S'
#         Store the results back into the req_date column (i.e., overwrite the old column values).


In [7]:
# Examine the results of the above operation by doing a head() check.


In [9]:
# Do an info() check as well.


<br><br>
### Get a picture of which IP addresses made the most requests.  Do this by using groupby() capabilities.

In [10]:
# Step 4. From a security perspective, let's check if any IP addresses seem to be making a large number of requests.
#         Do this by grouping by 'addr' and then getting the size() of each group.  Take the 10 largest groups by
#         doing .nlargest(10).  Print the top 10 addresses.


<br><br>
### Plot these results in a 'barh' graph.

In [11]:
# Plot the results directly from the Series obtained above.  Use the .plot() method supplying a kind='barh' parameter.


<br><br>
### Create a new column containing only the HTTP Method (e.g., 'GET', 'POST', etc.)

In [12]:
# Step 5. POST requests might be more of a security concern than GET requests.  Let's extract the first word
#         out of the 'path' column and place that into a new column called 'method'
#         Select the path column, and then use apply().  Pass into it a lambda function.  This function will 
#         be called for every row in the DataFrame.  The lambda looks like this:     lambda cell: cell.split()[0]

#         Save the results of this operation into the new column as follows:    log['method'] = log.path.apply()


In [13]:
# Do a head() check again.


<br><br>
### Examine only the 'POST' records, and get the top 10 addresses making 'POST' requests

In [14]:
# Step 6. With the new 'method' column, use a mask to only consider 'POST' requests now.  Plug the mask back into the
#         array and then groupby() 'addr' again.  Get the size() of each group and keep the top 10 (.nlargest(10) ).


<br><br>
### Plot these results in a 'barh' graph again.

In [15]:
# As before, plot the results from the Series obtained above using .plot() and supplying a kind='barh' parameter.


<br><br>
### Lookup IP addresses retrieving city, country, latitude, and longitude information for each one

In [21]:
ip_addr_lookup_file = '../resources/GeoLiteCity.dat'

In [22]:
# Step 7. For fun, let's look up some of these IP addresses.  For example, pass the most common IP address
#         into a call to geoip.record_by_addr() in the space below.  (Note: it was '149.56.83.40')
geoip = GeoIP(ip_addr_lookup_file)


In [23]:
def get_location(addr):
    results = ['', '', 0, 0]
    try:
        info = geoip.record_by_addr(addr)
        if info:
            results = [info.get('country_name'), info.get('city'), info.get('latitude'), info.get('longitude')]
    except GeoIPError:
        pass
    
    return results

In [20]:
# Now we'll repeat the lookup for EVERY record in the DataFrame.  A function has been provided above that accepts an
# address and returns a list of [country, city, latitude, longitude].  Pass that function into the apply() call below.
# Note: running this cell may take 1-3 minutes.
results = log.addr.apply()

In [21]:
# The results from the lookup are converted to a DataFrame below.  Run this cell.
locations = pd.DataFrame(results.to_numpy().tolist(), columns=['country', 'city', 'latitude', 'longitude'])

In [24]:
# Perform a head() check on the locations DataFrame.


<br><br>
### Join the original DataFrame and the new results together using concat()

In [25]:
# Step 8. Join the original DataFrame (log) to the new DataFrame (locations) in a row-by-row manner.
#         To do this, use pd.concat() passing in a list of DataFrames [log, locations] and perform the operation
#         along axis=1 (columns direction).  Call the new DataFrame "merged".  merged = pd.concat()  
#         View the first few records again.


<br><br>
### Plot the top 10 cities identified within the newly merged DataFrame.  Hint: *.groupby('city')*

In [26]:
# Step 9. Group the merged DataFrame by 'city' and get the size() of each group and take the .nlargest(10) again.
#         Plot the results in a 'barh' graph again.


<br><br>
### Plot these cities in a world map using the Folium library
For more on Folium, visit: https://pypi.org/project/folium/

In [ ]:
# For fun, let's plot these top 10 cities on a map.  Below, the code extracts the latitudes and longitudes of
# the top 10 cities.  Verify this by running this cell and printing the lat_longs variable.
lat_longs = []
for city in top10_cities.index:
    latitude = merged.loc[city == merged['city'], 'latitude']
    longitude = merged.loc[city == merged['city'], 'longitude']
    lat_longs.append((latitude.iloc[0], longitude.iloc[0]))


In [ ]:
# Pass lat_longs into the HeatMap() object below and run this cell
map = Map(location=[40, -60], tiles="OpenStreetMap", zoom_start=3)
hm = plugins.HeatMap(       , radius=15, blur=10)
hm.add_to(map)
map